In [27]:
using PyCall
using Random
using LinearAlgebra
using MLDataPattern
using Printf
using DataFrames
using CSV
using ProgressMeter
using JLD
using Flux
using ScikitLearn
using StatsBase
using TickTock

In [34]:
rate = 3e-5   # learning rate
batchsize = 100  # batch size
epochs = 1    # number of epochs
percent = 0.1
type = 3
k = 25
hidden_layer = 8000
hidden_layer2 = 5000
hidden_layer3 = 2000
hidden_layer4 = 1000
hidden_layer5 = 800
hidden_layer6 = 500
hidden_layer7 = 100
hidden_layer8 = 50
hidden_layer9 = 20
hidden_layer10 = 10

@time structtype = @sprintf(" dssp%s", type)
k = @sprintf("k%s", k)
tick()
println("start netsurf")
netsurf = CSV.read("../data/netsurf.csv", DataFrame) # read dataset
select!(netsurf, Not(" cb513_mask")) # remove column
for i in ["input", " dssp3", " dssp8"] # remove white spaces in strings
    netsurf[!, i] = [join(map(x -> isspace(a[x]) ? "" : a[x], 1:length(a))) for a in netsurf[!, i]]
end
seq_list = netsurf.input
println("netsurf loaded")

classes = union(hcat([Set(i) for i in netsurf[!, @sprintf(" dssp%s", type)]])...)
classes_idx = Dict(zip(classes, [i for i in 1:length(classes)]))

println("load data")
x = JLD.load(@sprintf("../data/data_netsurf/%s0.jld", k))["k25[ranges[i]:ranges[i + 1]]"]
xl = length(x)
less_data = StatsBase.sample([i for i in 1:xl], Int(round(percent*xl)), replace = false, ordered = true)
x = [x[i] for i in less_data]
x = [j for i in x for j in i]
x = hcat(x...)
x = convert.(Float32, x)

  0.000015 seconds (6 allocations: 264 bytes)
start netsurf


┌ Info:  started timer at: 2023-05-07T09:38:21.077
└ @ TickTock /home/mfat/.julia/packages/TickTock/fGILW/src/TickTock.jl:54


netsurf loaded


load data


10000×3974 Matrix{Float32}:
 0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  …  0.0  1.0  1.0  1.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0     1.0  1.0  0.0  1.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0     1.0  0.0  0.0  0.0  0.0  1.0  0.0
 1.0  1.0  0.0  0.0  0.0  1.0  0.0  1.0     0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  1.0  0.0  1.0  1.0     1.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0  …  1.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0     0.0  0.0  1.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0     1.0  0.0  0.0  0.0  1.0  1.0  1.0
 0.0  1.0  0.0  0.0  0.0  1.0  1.0  1.0     1.0  1.0  0.0  1.0  0.0  1.0  0.0
 0.0  0.0  1.0  1.0  1.0  0.0  0.0  0.0     0.0  1.0  0.0  0.0  1.0  0.0  0.0
 ⋮                        ⋮              ⋱                 ⋮              
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0     0.0  1.0  0.0  0.0  1.0  1.0  0.0
 1.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0

In [35]:
y = first(netsurf[!, structtype], xl)
y = [y[i] for i in less_data]
y = [j for i in y for j in i]

groups = []
seq_list = [netsurf.input[i] for i in less_data]
for (i,seq) in enumerate(seq_list)
    group = [i for j in seq]
    global groups = vcat(groups,group)
end

In [36]:
pyx = PyObject([i for i in 1:length(groups)])
pygroups = PyObject(groups)
x = PyObject(x)
y = PyObject(split(join(y, ""), ""))
pyrate = PyObject(rate)
pyepochs = PyObject(Int(epochs))
pyhl = PyObject(hidden_layer)
pyhl2 = PyObject(hidden_layer2)
pyhl3 = PyObject(hidden_layer3)
pyhl4 = PyObject(hidden_layer4)
pyhl5 = PyObject(hidden_layer5)
pyhl6 = PyObject(hidden_layer6)
pyhl7 = PyObject(hidden_layer7)
pyhl8 = PyObject(hidden_layer8)
pyhl9 = PyObject(hidden_layer9)
pyhl10 = PyObject(hidden_layer10)
pybs = PyObject(batchsize)
pyclasses = PyObject(split(join(collect(classes), ""), ""))
pytype = PyObject(type)
pyk = PyObject(k)
pyper = PyObject(percent)

PyObject 0.1

In [31]:
py"""
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

groups = $pygroups

$x = torch.from_numpy($x.T).to(torch.float32)

encoder = LabelEncoder()
encoder.fit($pyclasses)
y_train = encoder.transform($y)
"""

In [32]:
py"""
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        if self.transform:
            x = self.transform(x)
        return x, y

train_dataset = CustomDataset(x_train, y_train, transform=None)
test_dataset = CustomDataset(x_test, y_test, transform=None)
train_loader = DataLoader(train_dataset, batch_size=$pybs, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=$pybs, num_workers=8)

class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, hidden_dim6, hidden_dim7, hidden_dim8, hidden_dim9, hidden_dim10, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, hidden_dim5)
        self.fc6 = torch.nn.Linear(hidden_dim5, hidden_dim6)
        self.fc7 = torch.nn.Linear(hidden_dim6, hidden_dim7)
        self.fc8 = torch.nn.Linear(hidden_dim7, hidden_dim8)
        self.fc9 = torch.nn.Linear(hidden_dim8, hidden_dim9)
        self.fc10 = torch.nn.Linear(hidden_dim9, hidden_dim10)
        self.fc11 = torch.nn.Linear(hidden_dim10, output_dim)
        self.activation = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.fc8(x)
        x = self.activation(x)
        x = self.fc9(x)
        x = self.activation(x)
        x = self.fc10(x)
        x = self.activation(x)
        x = self.fc11(x)
        return x
"""

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'NameError'>
NameError("name 'x_train' is not defined")
  File "/home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 18, in <module>
    ns["__builtin__"] = builtin


In [17]:
py"""
input_dim = 10000
hidden_dim = $pyhl
hidden_dim2 = $pyhl2
hidden_dim3 = $pyhl3
hidden_dim4 = $pyhl4
hidden_dim5 = $pyhl5
hidden_dim6 = $pyhl6
hidden_dim7 = $pyhl7
hidden_dim8 = $pyhl8
hidden_dim9 = $pyhl9
hidden_dim10 = $pyhl10
output_dim = $pytype
num_epochs = $pyepochs

model = torch.MLP(input_dim, hidden_dim, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, hidden_dim6, hidden_dim7, hidden_dim8, hidden_dim9, hidden_dim10, output_dim)
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=$pyrate)
"""

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'NameError'>
NameError("name 'MLP' is not defined")
  File "/home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 15, in <module>
    # http://mail.python.org/pipermail/python-dev/2001-April/014068.html


In [18]:
using Conda
Conda.add("pickle")

┌ Info: Running `conda install -y pickle` in root environment
└ @ Conda /home/mfat/.julia/packages/Conda/kOnIE/src/Conda.jl:127


Retrieving notices: ...working... 

done


done
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

done
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - pickle

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




ProcessFailedException: failed process: Process(setenv(`/home/mfat/.julia/conda/3/x86_64/bin/conda install -y pickle`,["PATH=/app/bin:/app/bin:/app/bin:/usr/bin:/home/mfat/.var/app/com.vscodium.codium/data/node/bin:/home/mfat/.var/app/com.vscodium.codium/data/cargo/bin:/home/mfat/.var/app/com.vscodium.codium/data/python/bin:/home/mfat/.var/app/com.vscodium.codium/data/gem/ruby/3.1.0/bin", "ZYPAK_SPAWN_LATEST_ON_REEXEC=0", "ALSA_CONFIG_DIR=/usr/share/alsa", "CPU=x86_64", "JRE_HOME=/usr/lib64/jvm/java-11-openjdk-11", "FROM_HEADER=", "VSCODE_NLS_CONFIG={\"locale\":\"en-us\",\"osLocale\":\"en\",\"availableLanguages\":{},\"_languagePackSupport\":true}", "G_FILENAME_ENCODING=@locale,UTF-8,ISO-8859-15,CP1252", "XSESSION_IS_UP=yes", "NO_AT_BRIDGE=1", "XNLSPATH=/usr/share/X11/nls", "LD_LIBRARY_PATH=/app/lib", "DBUS_SESSION_BUS_ADDRESS=unix:path=/run/flatpak/bus", "VDPAU_DRIVER=va_gl", "MANPATH=/home/mfat/.local/share/man:/usr/local/man:/usr/share/man", "XDG_SEAT_PATH=/org/freedesktop/DisplayMan

In [19]:
py"""
loss_list = []
for epoch in range($pyepochs):
    train_loss = 0.0

    # Train model
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Evaluate model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            target_onehot = torch.nn.functional.one_hot(target, num_classes=output_dim).float()
            output = model(data)
            pred = output.argmax(dim=1)
            y_true.extend(target.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())

    # Compute evaluation metrics
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)

    print(f"Epoch {epoch+1}/{num_epochs}: train_loss={train_loss/len(train_loader):.4f}, acc={acc:.4f}, f1={f1:.4f}", flush=True)
    loss_list.append(train_loss)
    print("Confusion matrix:", flush=True)
    print(cm, flush=True)
print(loss_list/len(train_loader), flush = True)
filename = "data/MLP_GPU_bit_%s_ep%s_lr%s_%s_dssp%s_%s_dp%s" % ($pyk, $pyepochs, $pyrate, "ultra", $pytype, $pyper)
torch.save(model.state_dict(), filename)
"""


PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'NameError'>
NameError("name 'model' is not defined")
  File "/home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 6, in <module>


In [41]:
py"""
print(y_train, flush=True)
print(len(train_loader), flush=True)
"""

[0 0 0 ... 0 0 0]
40


In [46]:
py"""
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

groups = $pygroups
print("prob1", flush=True)
x_train = torch.from_numpy($x.T)

encoder = LabelEncoder()
encoder.fit($pyclasses)
y_train = encoder.transform($y)

from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, index):
        x = self.X[index]
        y = self.y[index]
        if self.transform:
            x = self.transform(x)
        return x, y

train_dataset = CustomDataset(x_train, y_train, transform=None)
train_loader = DataLoader(train_dataset, batch_size=$pybs, num_workers=8)

class MLP(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, hidden_dim6, hidden_dim7, hidden_dim8, hidden_dim9, hidden_dim10, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim2)
        self.fc3 = torch.nn.Linear(hidden_dim2, hidden_dim3)
        self.fc4 = torch.nn.Linear(hidden_dim3, hidden_dim4)
        self.fc5 = torch.nn.Linear(hidden_dim4, hidden_dim5)
        self.fc6 = torch.nn.Linear(hidden_dim5, hidden_dim6)
        self.fc7 = torch.nn.Linear(hidden_dim6, hidden_dim7)
        self.fc8 = torch.nn.Linear(hidden_dim7, hidden_dim8)
        self.fc9 = torch.nn.Linear(hidden_dim8, hidden_dim9)
        self.fc10 = torch.nn.Linear(hidden_dim9, hidden_dim10)
        self.fc11 = torch.nn.Linear(hidden_dim10, output_dim)
        self.activation = torch.nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.fc8(x)
        x = self.activation(x)
        x = self.fc9(x)
        x = self.activation(x)
        x = self.fc10(x)
        x = self.activation(x)
        x = self.fc11(x)
        return x

input_dim = 10000
hidden_dim = $pyhl
hidden_dim2 = $pyhl2
hidden_dim3 = $pyhl3
hidden_dim4 = $pyhl4
hidden_dim5 = $pyhl5
hidden_dim6 = $pyhl6
hidden_dim7 = $pyhl7
hidden_dim8 = $pyhl8
hidden_dim9 = $pyhl9
hidden_dim10 = $pyhl10
output_dim = $pytype
num_epochs = $pyepochs

model = MLP(input_dim, hidden_dim, hidden_dim2, hidden_dim3, hidden_dim4, hidden_dim5, hidden_dim6, hidden_dim7, hidden_dim8, hidden_dim9, hidden_dim10, output_dim)
print("prob2", flush=True)
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=$pyrate)

loss_list = []
for epoch in range($pyepochs):
    train_loss = 0.0

    # Train model
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        print(data, flush=True)
        print(target, flush=True)
        print("prob3", flush=True)
        data, target = data.to(device), target.to(device)
        print("prob4", flush=True)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    print(epoch+1/num_epochs, flush=True)
    print(train_loss/len(train_loader), flush=True)
    loss_list.append(train_loss)
loss_list = [i/len(train_loader) for i in loss_list]
print(loss_list, flush = True)
filename = "data/MLP_GPUfull_bit_%s_ep%s_lr%s_dssp%s_p%s" % ($pyk, $pyepochs, $pyrate, $pytype, $pyper)
torch.save(model.state_dict(), filename)
"""
tock()

prob1
prob2
tensor([[0., 0., 0.,  ..., 1., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 1., 0.],
        ...,
        [0., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1])
prob3
prob4
tensor([[0., 0., 1.,  ..., 0., 0., 1.],
        [0., 1., 1.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 1., 0.],
        [0., 1., 1.,  ..., 0., 1., 1.],
        [0., 1., 0.,  ..., 1., 1., 1.]])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl:38 =# @pysym(:PyEval_EvalCode), PyPtr, (PyPtr, PyPtr, PyPtr), o, globals, locals))))) <class 'RuntimeError'>
RuntimeError('Parent directory data does not exist.')
  File "/home/mfat/.julia/packages/PyCall/twYvK/src/pyeval.jl", line 125, in <module>
    i += 1
  File "/home/mfat/.julia/conda/3/x86_64/lib/python3.10/site-packages/torch/serialization.py", line 440, in save
    with _open_zipfile_writer(f) as opened_zipfile:
  File "/home/mfat/.julia/conda/3/x86_64/lib/python3.10/site-packages/torch/serialization.py", line 315, in _open_zipfile_writer
    return container(name_or_buffer)
  File "/home/mfat/.julia/conda/3/x86_64/lib/python3.10/site-packages/torch/serialization.py", line 288, in __init__
    super().__init__(torch._C.PyTorchFileWriter(str(name)))
